# 대리점 데이터를 가공하는 테크닉

#### 데이터 읽기

In [1]:
import pandas as pd

In [2]:
uriage = pd.read_csv("./pyda100-master/2장/uriage.csv")
uriage

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현
...,...,...,...,...
2994,2019-02-15 2:56,상품Y,2500.0,김정민
2995,2019-06-22 4:03,상품M,1300.0,김재원
2996,2019-03-29 11:14,상품Q,NaN,김지율
2997,2019-07-14 12:56,상품H,NaN,김승주


In [3]:
kokyaku_daicho = pd.read_excel("./pyda100-master/2장/kokyaku_daicho.xlsx")
kokyaku_daicho

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127
...,...,...,...
195,김 재희,G시,2017-06-20 00:00:00
196,김 도영,E시,2018-06-20 00:00:00
197,김 이안,F시,2017-04-29 00:00:00
198,김 시현,H시,2019-04-19 00:00:00


#### 데이터 오류를 살펴보자.

In [4]:
uriage['item_name'].head(10)

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
5      상품S
6     상품 a
7      상품V
8      상품O
9      상품A
Name: item_name, dtype: object

In [5]:
uriage['purchase_date'] = pd.to_datetime(uriage['purchase_date'])
uriage['purchase_month'] = uriage['purchase_date'].dt.strftime("%Y-%m")
uriage.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019-01,1400,0,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
2019-02,0,0,0,0,0,0,2400,0,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
2019-03,0,500,1300,1600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-04,0,0,0,0,0,0,0,2300,0,1700,...,0,0,0,0,0,1900,0,0,0,0
2019-05,0,0,0,0,1900,0,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
2019-06,0,0,0,0,0,2300,0,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
2019-07,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


#### 오류 수정(정합성)

In [6]:
len(uriage['item_name'].unique()) #26

99

In [7]:
uriage['item_name'] = uriage['item_name'].str.upper()
uriage['item_name'] = uriage['item_name'].str.replace(" ", "")
uriage['item_name'].head()

0    상품A
1    상품S
2    상품A
3    상품Z
4    상품A
Name: item_name, dtype: object

In [8]:
len(uriage['item_name'].unique())

26

In [9]:
uriage['item_name'].unique()

array(['상품A', '상품S', '상품Z', '상품V', '상품O', '상품U', '상품L', '상품C', '상품I',
       '상품R', '상품X', '상품G', '상품P', '상품Q', '상품Y', '상품N', '상품W', '상품E',
       '상품K', '상품B', '상품F', '상품D', '상품M', '상품H', '상품T', '상품J'],
      dtype=object)

#### 금액의 결측치 수정

In [10]:
uriage.isnull().sum()

purchase_date       0
item_name           0
item_price        387
customer_name       0
purchase_month      0
dtype: int64

In [11]:
len(uriage.item_price.unique())

27

In [12]:
uriage.item_price.unique()

array([ 100.,   nan, 2600., 1900., 2200., 1500., 2100., 1200.,  300.,
        900., 1800., 2400.,  700., 1600., 1700., 2500., 2300.,  500.,
       1100.,  200.,  600., 1400.,  400., 1300.,  800., 2000., 1000.])

In [13]:
flg_is_null = uriage['item_price'].isnull()
for trg in list(uriage.loc[flg_is_null, 'item_name'].unique()) : 
    price = uriage.loc[(~flg_is_null) & (uriage['item_name'] == trg), 'item_price'].max()
    uriage['item_price'].loc[(flg_is_null) & (uriage['item_name'] == trg)] = price
uriage.head()

C:\Users\jsj26\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,2019-06
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,2019-07
2,2019-05-11 19:42:00,상품A,100.0,김유찬,2019-05
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,2019-02
4,2019-04-22 03:09:00,상품A,100.0,김강현,2019-04


In [14]:
len(uriage.item_price.unique())

26

In [15]:
uriage.item_price.unique()

array([ 100., 1900., 2600., 2200., 1500., 2100., 1200.,  300.,  900.,
       1800., 2400.,  700., 1600., 1700., 2500., 1400., 2300.,  500.,
       1100.,  200.,  600.,  400., 1300.,  800., 2000., 1000.])

In [16]:
uriage.isnull().sum()[uriage.isnull().sum()>0]

Series([], dtype: int64)

In [17]:
tmp = pd.DataFrame(uriage.groupby('item_name')['item_price'].min()) #skipna=False 옵션을 사용을 못해서.. 이 방법은 X
tmp = pd.merge(tmp, pd.DataFrame(uriage.groupby('item_name')['item_price'].max()), on='item_name')
tmp

,item_price_x,item_price_y
item_name,,
상품A,100.0,100.0
상품B,200.0,200.0
상품C,300.0,300.0
상품D,400.0,400.0
상품E,500.0,500.0
상품F,600.0,600.0
상품G,700.0,700.0
상품H,800.0,800.0
상품I,900.0,900.0


In [18]:
for trg in list(uriage['item_name'].sort_values().unique()) :
    print(trg + "\t최고가 : " + str(uriage.loc[uriage['item_name']==trg]['item_price'].max()) + "\t최저가 : " + str(uriage.loc[uriage['item_name']==trg]['item_price'].min(skipna=False)))

상품A	최고가 : 100.0	최저가 : 100.0
상품B	최고가 : 200.0	최저가 : 200.0
상품C	최고가 : 300.0	최저가 : 300.0
상품D	최고가 : 400.0	최저가 : 400.0
상품E	최고가 : 500.0	최저가 : 500.0
상품F	최고가 : 600.0	최저가 : 600.0
상품G	최고가 : 700.0	최저가 : 700.0
상품H	최고가 : 800.0	최저가 : 800.0
상품I	최고가 : 900.0	최저가 : 900.0
상품J	최고가 : 1000.0	최저가 : 1000.0
상품K	최고가 : 1100.0	최저가 : 1100.0
상품L	최고가 : 1200.0	최저가 : 1200.0
상품M	최고가 : 1300.0	최저가 : 1300.0
상품N	최고가 : 1400.0	최저가 : 1400.0
상품O	최고가 : 1500.0	최저가 : 1500.0
상품P	최고가 : 1600.0	최저가 : 1600.0
상품Q	최고가 : 1700.0	최저가 : 1700.0
상품R	최고가 : 1800.0	최저가 : 1800.0
상품S	최고가 : 1900.0	최저가 : 1900.0
상품T	최고가 : 2000.0	최저가 : 2000.0
상품U	최고가 : 2100.0	최저가 : 2100.0
상품V	최고가 : 2200.0	최저가 : 2200.0
상품W	최고가 : 2300.0	최저가 : 2300.0
상품X	최고가 : 2400.0	최저가 : 2400.0
상품Y	최고가 : 2500.0	최저가 : 2500.0
상품Z	최고가 : 2600.0	최저가 : 2600.0


#### 고객 이름 오류 수정

In [19]:
kokyaku_daicho['고객이름'] = kokyaku_daicho['고객이름'].str.replace(" ", '')
kokyaku_daicho['고객이름'].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

#### 날짜 오류 수정 - 등록일

In [20]:
kokyaku_daicho.head()

,고객이름,지역,등록일
0,김현성,H시,2018-01-04 00:00:00
1,김도윤,E시,42782
2,김지한,A시,2018-01-07 00:00:00
3,김하윤,F시,42872
4,김시온,E시,43127


In [21]:
kokyaku_daicho[kokyaku_daicho['등록일'].astype('str').str.isdigit()]

,고객이름,지역,등록일
1,김도윤,E시,42782
3,김하윤,F시,42872
4,김시온,E시,43127
21,김동우,A시,42920
27,김서준,C시,42901
47,김태민,H시,42741
49,김정원,H시,42929
53,김주영,E시,42833
76,김지성,A시,43188
80,김지후,D시,43110


In [22]:
flg_is_serial = kokyaku_daicho['등록일'].astype('str').str.isdigit()
flg_is_serial.sum()

22

#### datetime 모듈의 데이터 유형
import datetime
- datetime.date(년, 월, 일)
- datetime.time(시, 분, 초, 마이크로초)
- datetime.datetime(년, 월, 일, 시, 분, 초, 마이크로초)
- datetime.timedelta(일, 초, 마이크로초) : 두 datetime 값의 차이를 나타낸다.

In [23]:
fromSerial = pd.to_timedelta(kokyaku_daicho.loc[flg_is_serial, "등록일"].astype('float')-2, unit="D") + pd.to_datetime("1900/01/01") #엑셀에서 날짜를 읽어 들일 때 2일이 차이나게 됨.  
# unit ="D" : 단위가 day
fromSerial

1     2017-02-16
3     2017-05-17
4     2018-01-27
21    2017-07-04
27    2017-06-15
47    2017-01-06
49    2017-07-13
53    2017-04-08
76    2018-03-29
80    2018-01-10
99    2017-05-30
114   2018-06-03
118   2018-01-29
122   2018-04-16
139   2017-05-25
143   2017-03-24
155   2017-01-19
172   2018-03-22
179   2017-01-08
183   2017-07-24
186   2018-07-13
192   2018-06-08
Name: 등록일, dtype: datetime64[ns]

In [24]:
fromString = pd.to_datetime(kokyaku_daicho.loc[~flg_is_serial, "등록일"])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [25]:
kokyaku_daicho['등록일'] = pd.concat([fromSerial, fromString]) #인덱스가 있어서 인덱스대로 concat됨
kokyaku_daicho

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-16
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-17
4,김시온,E시,2018-01-27
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


In [26]:
kokyaku_daicho['등록연월'] = kokyaku_daicho['등록일'].dt.strftime("%Y%m")
len(kokyaku_daicho['등록연월'])

200

In [27]:
flg_is_serial = kokyaku_daicho['등록일'].astype('str').str.isdigit()
flg_is_serial.sum()

0

#### 고객 이름을 키로 두 개의 데이터를 조인. 

In [28]:
# 이름을 키로 조인하는 것은 좋지 않지만 회원 이름 간에 중복이 없으므로 이 데이터에서 키로 사용해도 좋다. 
len(kokyaku_daicho['고객이름'].unique())

200

In [29]:
join_data = pd.merge(uriage, kokyaku_daicho, left_on="customer_name", right_on="고객이름", how="left")
join_data = join_data.drop("customer_name", axis=1)
join_data

,purchase_date,item_name,item_price,purchase_month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,2019-06,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,2019-07,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,2019-05,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,2019-02,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,2019-04,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,2019-02,김정민,B시,2017-07-01,201707
2995,2019-06-22 04:03:00,상품M,1300.0,2019-06,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14:00,상품Q,1700.0,2019-03,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56:00,상품H,800.0,2019-07,김승주,E시,2018-07-13,201807


#### 정제한 데이터 덤프

In [30]:
dump_data = join_data[['purchase_date', 'purchase_month', 'item_name', 'item_price', '고객이름', '지역', '등록일']]
dump_data.to_csv("./pyda100-master/2장/dump_data.csv", index=False)

#### 데이터 집계

In [31]:
import_data = pd.read_csv("./pyda100-master/2장/dump_data.csv")
import_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,2019-06,상품A,100.0,김가온,C시,2017-01-26 00:00:00
1,2019-07-13 13:05:00,2019-07,상품S,1900.0,김우찬,C시,2018-04-07 00:00:00
2,2019-05-11 19:42:00,2019-05,상품A,100.0,김유찬,A시,2018-06-19 00:00:00
3,2019-02-12 23:40:00,2019-02,상품Z,2600.0,김재현,D시,2018-07-22 00:00:00
4,2019-04-22 03:09:00,2019-04,상품A,100.0,김강현,D시,2017-06-07 00:00:00
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,2019-02,상품Y,2500.0,김정민,B시,2017-07-01 00:00:00
2995,2019-06-22 04:03:00,2019-06,상품M,1300.0,김재원,E시,2018-03-31 00:00:00
2996,2019-03-29 11:14:00,2019-03,상품Q,1700.0,김지율,B시,2017-03-15 00:00:00
2997,2019-07-14 12:56:00,2019-07,상품H,800.0,김승주,E시,2018-07-13 00:00:00


In [32]:
byItem = import_data.pivot_table(index="purchase_month", columns="item_name", aggfunc="size", fill_value=0)
byItem

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019-01,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
2019-02,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
2019-03,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
2019-04,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
2019-05,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
2019-06,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
2019-07,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [33]:
byPrice = import_data.pivot_table(index="purchase_month", columns="item_name", values="item_price", aggfunc="sum", fill_value=0)
byPrice

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019-01,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
2019-02,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
2019-03,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
2019-04,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
2019-05,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
2019-06,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
2019-07,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [34]:
byCustomer = import_data.pivot_table(index="purchase_month", columns="고객이름", aggfunc="size", fill_value=0)
byCustomer

고객이름,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,...,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019-01,1,2,1,2,5,3,1,1,1,5,...,3,2,4,2,1,1,4,4,4,3
2019-02,3,1,4,3,3,5,2,3,1,5,...,5,3,7,0,0,1,2,0,2,2
2019-03,4,0,3,1,1,2,2,1,1,5,...,3,2,1,2,1,2,1,2,3,4
2019-04,1,0,3,0,2,1,2,6,2,4,...,0,4,3,1,1,2,2,4,3,1
2019-05,0,0,2,2,1,4,6,3,3,1,...,2,2,1,4,3,0,0,0,1,2
2019-06,5,0,3,0,2,1,6,1,1,1,...,2,4,1,0,4,2,2,0,0,2
2019-07,3,1,3,2,2,1,4,1,4,3,...,2,1,0,4,0,2,6,1,2,3


In [35]:
byRegion = import_data.pivot_table(index="purchase_month", columns="지역", aggfunc="size", fill_value=0)
byRegion

지역,A시,B시,C시,D시,E시,F시,G시,H시
purchase_month,,,,,,,,
2019-01,59,55,72,34,49,57,49,42
2019-02,71,46,65,48,61,52,43,63
2019-03,64,52,57,43,52,59,51,59
2019-04,64,48,54,45,48,58,40,52
2019-05,57,52,68,48,59,65,35,43
2019-06,53,47,61,30,51,51,58,58
2019-07,76,53,61,42,54,64,47,54


#### 집계기간에 구매 이력이 없는 사용자

In [36]:
away_data = pd.merge(uriage, kokyaku_daicho, left_on="customer_name", right_on="고객이름", how="right")
away_data[away_data["purchase_date"].isnull()][['고객이름', '등록일']]

,고객이름,등록일
2999,김서우,2019-04-23
